In [ ]:
"""
Your task in this exercise has two steps:

- audit the OSMFILE and change the variable 'mapping' to reflect the changes needed to fix 
    the unexpected street types to the appropriate ones in the expected list.
    You have to add mappings only for the actual problems you find in this OSMFILE,
    not a generalized solution, since that may and will depend on the particular area you are auditing.
- write the update_name function, to actually fix the street name.
    The function takes a string with street name as an argument and should return the fixed name
    We have provided a simple test so that you see what exactly is expected
"""
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = "example.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

# UPDATE THIS VARIABLE
mapping = { "St": "Street",
            "St.": "Street",
            "Rd.": "Road",
            "Ave": "Avenue",
            "Rd": "Road"
            }


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types


def update_name(name, mapping):

    # YOUR CODE HERE
    for key in mapping.keys():
        if key in name:
            t = name.split(" ")
            t[-1] = mapping[key]
            name = ""
            for s in t:
                name += " " + s
    return name.strip()


def test():
    st_types = audit(OSMFILE)
    assert len(st_types) == 3
    pprint.pprint(dict(st_types))

    for st_type, ways in st_types.iteritems():
        for name in ways:
            better_name = update_name(name, mapping)
            print name, "=>", better_name
            if name == "West Lexington St.":
                assert better_name == "West Lexington Street"
            if name == "Baldwin Rd.":
                assert better_name == "Baldwin Road"


if __name__ == '__main__':
    test()

In [30]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import xml.etree.cElementTree as ET
import pprint
import re
import codecs
import json
"""
Your task is to wrangle the data and transform the shape of the data
into the model we mentioned earlier. The output should be a list of dictionaries
that look like this:

{
"id": "2406124091",
"type: "node",
"visible":"true",
"created": {
          "version":"2",
          "changeset":"17206049",
          "timestamp":"2013-08-03T16:43:42Z",
          "user":"linuxUser16",
          "uid":"1219059"
        },
"pos": [41.9757030, -87.6921867],
"address": {
          "housenumber": "5157",
          "postcode": "60625",
          "street": "North Lincoln Ave"
        },
"amenity": "restaurant",
"cuisine": "mexican",
"name": "La Cabana De Don Luis",
"phone": "1 (773)-271-5176"
}

You have to complete the function 'shape_element'.
We have provided a function that will parse the map file, and call the function with the element
as an argument. You should return a dictionary, containing the shaped data for that element.
We have also provided a way to save the data in a file, so that you could use
mongoimport later on to import the shaped data into MongoDB. 

Note that in this exercise we do not use the 'update street name' procedures
you worked on in the previous exercise. If you are using this code in your final
project, you are strongly encouraged to use the code from previous exercise to 
update the street names before you save them to JSON. 

In particular the following things should be done:
- you should process only 2 types of top level tags: "node" and "way"
- all attributes of "node" and "way" should be turned into regular key/value pairs, except:
    - attributes in the CREATED array should be added under a key "created"
    - attributes for latitude and longitude should be added to a "pos" array,
      for use in geospacial indexing. Make sure the values inside "pos" array are floats
      and not strings. 
- if the second level tag "k" value contains problematic characters, it should be ignored
- if the second level tag "k" value starts with "addr:", it should be added to a dictionary "address"
- if the second level tag "k" value does not start with "addr:", but contains ":", you can
  process it in a way that you feel is best. For example, you might split it into a two-level
  dictionary like with "addr:", or otherwise convert the ":" to create a valid key.
- if there is a second ":" that separates the type/direction of a street,
  the tag should be ignored, for example:

<tag k="addr:housenumber" v="5158"/>
<tag k="addr:street" v="North Lincoln Avenue"/>
<tag k="addr:street:name" v="Lincoln"/>
<tag k="addr:street:prefix" v="North"/>
<tag k="addr:street:type" v="Avenue"/>
<tag k="amenity" v="pharmacy"/>

  should be turned into:

{...
"address": {
    "housenumber": 5158,
    "street": "North Lincoln Avenue"
}
"amenity": "pharmacy",
...
}

- for "way" specifically:

  <nd ref="305896090"/>
  <nd ref="1719825889"/>

should be turned into
"node_refs": ["305896090", "1719825889"]
"""

SAMPLE_FILE = "sample.osm"
# lower = re.compile(r'^([a-z]|_)*$')
# lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
# problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
# startswith = re.compile(r'addr:')
POS = ["lon","lat"]

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

# UPDATE THIS VARIABLE
mapping = { "St": "Street",
            "St.": "Street",
            "Rd.": "Road",
            "Ave": "Avenue",
            "Rd": "Road"
            }
def update_name(name, mapping):

    # YOUR CODE HERE
    for key in mapping.keys():
        if key in name:
            t = name.split(" ")
            t[-1] = mapping[key]
            name = ""
            for s in t:
                name += " " + s
    return name.strip()

def audit_street_type(street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            better_name = update_name(street_type,mapping)
            return better_name
    else:
        return street_name
    
            
def shape_element(element):
    node = {}
    address = {}
    created = {}
    node_refs = []
    pos = []
    if element.tag == "node" or element.tag == "way" :
        # YOUR CODE HERE
        node['type'] = element.tag

        # As 'element.attrib' is a dictionary, you can search it for keys directly:
        if 'lat' in element.attrib.keys() and 'lon' in element.attrib.keys():
            try:
                lat = float(element.attrib['lat'])
                lon = float(element.attrib['lon'])
                pos.insert(0,lat)
                pos.insert(1,lon)
            # or pos = [lat,lon]
            except:
                pass
        for k, m in element.attrib.items():
            # you have taken care of 'lat' and 'lon' so skip those:
            if k not in POS:
                if k in CREATED:
                    created[k] = m
                else:
                    node[k] = m
    
        for child in element:
            if child.tag == "nd":
                #node["node_refs"].append(child.attrib['ref'])
                node_refs.append(child.attrib['ref'])

            elif child.tag == "tag":
                if child.attrib['k'] == 'addr:housenumber':
                # this is where the key/values of the dictionary are created
                    address['housenumber']=child.attrib['v'] 
                if child.attrib['k'] == 'addr:postcode':
                # this is where the key/values of the dictionary are created
                    if 'D' in child.attrib['v']:
                        return None
                    else:
                        address['postcode']=child.attrib['v']
                if child.attrib['k'] == 'addr:street':
                # this is where the key/values of the dictionary are created   
                    address['street']=audit_street_type(,child.attrib['v'])
        if created:
            node['created'] = created
        if pos:
            node['pos'] = pos
        if address:
            node['address'] = address
        if node_refs:
            node['node_refs'] = node_refs
        return node
    else:
        return None


def process_map(file_in, pretty = False):
    # You do not need to change this file
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data

def test():
    # NOTE: if you are running this code on your computer, with a larger dataset, 
    # call the process_map procedure with pretty=False. The pretty=True option adds 
    # additional spaces to the output, making it significantly larger.
    data = process_map(SAMPLE_FILE, False)
    #pprint.pprint(data)
    
#     correct_first_elem = {
#         "id": "261114295", 
#         "visible": "true", 
#         "type": "node", 
#         "pos": [41.9730791, -87.6866303], 
#         "created": {
#             "changeset": "11129782", 
#             "user": "bbmiller", 
#             "version": "7", 
#             "uid": "451048", 
#             "timestamp": "2012-03-28T18:31:23Z"
#         }
#     }
#     assert data[0] == correct_first_elem
#     assert data[-1]["address"] == {
#                                     "street": "West Lexington St.", 
#                                     "housenumber": "1412"
#                                       }
#     assert data[-1]["node_refs"] == [ "2199822281", "2199822390",  "2199822392", "2199822369", 
#                                     "2199822370", "2199822284", "2199822281"]

if __name__ == "__main__":
    test()

In [31]:
import json

def insert_data(data, db):

    # Your code here. Insert the data into a collection 'arachnid'
    db.openstreetmap.insert(data)
    


if __name__ == "__main__":
    
    from pymongo import MongoClient
    client = MongoClient("mongodb://localhost:27017")
    db = client.osm

    with open('sample.osm.json') as f:
        for line in f:
            data = json.loads(line)
            insert_data(data, db)
    

E:\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


{u'id': u'71532138', u'_id': ObjectId('5920fc9a0229a009ec582f15'), u'type': u'node', u'pos': [22.4340617, 114.028622], u'created': {u'uid': u'2748195', u'changeset': u'38293888', u'version': u'3', u'user': u'karitotp', u'timestamp': u'2016-04-04T15:30:59Z'}}


In [32]:

print db.openstreetmap.find_one()

{u'id': u'71532138', u'_id': ObjectId('5920fc9a0229a009ec582f15'), u'type': u'node', u'pos': [22.4340617, 114.028622], u'created': {u'uid': u'2748195', u'changeset': u'38293888', u'version': u'3', u'user': u'karitotp', u'timestamp': u'2016-04-04T15:30:59Z'}}


In [34]:
def print_doc_num(query):
    print db.test_collection.find(query).count()

print_doc_num({})

0


In [33]:
def print_doc_num(query):
    print db.openstreetmap.find(query).count()

print_doc_num({"type":"node"})

65904


In [35]:
print_doc_num({"type":"way"})

0
